In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import featuretools as ft

In [2]:
train= pd.read_csv("C:\\Users\\AJIT\\Desktop\\Data science\\Hackathons\\previous hack\\Club Mahindra DataOlympics\\train.csv")
test = pd.read_csv("C:\\Users\\AJIT\\Desktop\\Data science\\Hackathons\\previous hack\\Club Mahindra DataOlympics\\test.csv")

In [3]:
train.shape

(341424, 24)

In [4]:
train.tail()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,...,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
341419,a804d59b74b1d532a90740ded80dd05873bbef125bc349...,11/02/19,27/02/19,01/03/19,2,2,4,0,47,1,...,8.0,1,2,I,1,0095e00c6dae789d37628bd85697e53c423231337994c4...,E,A,c6f3ac57944a531490cd39902d0f777715fd005efac9a3...,5.918087
341420,7062c92a631731e7189cf3dddb20aceeae91a883e14229...,27/02/19,28/02/19,01/03/19,2,4,2,0,45,1,...,10.0,1,2,D,1,42d7b97ad0d345da57a4010157729286418481f3d7ad24...,E,A,c75cb66ae28d8ebc6eded002c28a8ba0d06d3a78c6b5cb...,8.186771
341421,18a5668c302c321dc2d42a48923ceff97af633c66d72ee...,27/02/19,27/02/19,28/02/19,3,3,3,0,45,1,...,10.0,4,3,H,1,f393011a27daf3d1676417f48f44ad9ed3541285e71209...,F,A,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,8.381939
341422,1cbe1611a122023d7e994b4cf51fcf28ce44acdbcabbfb...,27/02/19,28/02/19,01/03/19,1,3,5,0,47,2,...,8.0,2,3,B,1,b14649c29eb050f4e9b69c7a3afb5dfa00d12dabfd2e40...,D,A,6208ef0f7750c111548cf90b6ea1d0d0a66f6bff40dbef...,8.856035
341423,8712dd3ce2e36f2fbcd16b24aa3b89bd1c5203a39e578b...,28/02/19,01/03/19,02/03/19,3,3,2,0,45,2,...,8.0,7,5,B,2,2a3936f649eb6af2167f41fb6f0cc9f352d142125f402f...,C,A,9400f1b21cb527d7fa3d3eabba93557a18ebe7a2ca4e47...,7.864627


In [5]:
train.shape

(341424, 24)

In [6]:
train.dtypes

reservation_id                         object
booking_date                           object
checkin_date                           object
checkout_date                          object
channel_code                            int64
main_product_code                       int64
numberofadults                          int64
numberofchildren                        int64
persontravellingid                      int64
resort_region_code                      int64
resort_type_code                        int64
room_type_booked_code                   int64
roomnights                              int64
season_holidayed_code                 float64
state_code_residence                  float64
state_code_resort                       int64
total_pax                               int64
member_age_buckets                     object
booking_type_code                       int64
memberid                               object
cluster_code                           object
reservationstatusid_code          

In [7]:
# saving identifiers
#test_Item_Identifier = test['Item_Identifier']
#test_Outlet_Identifier = test['Outlet_Identifier']
amount_spent_per_room_night_scaled = train['amount_spent_per_room_night_scaled']
train.drop(['amount_spent_per_room_night_scaled'], axis=1, inplace=True)

In [8]:
combi = train.append(test, ignore_index=True)

In [9]:
combi.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,...,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,...,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,...,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,...,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,...,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,...,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...


In [10]:
combi.shape

(488189, 23)

In [11]:
combi.isnull().sum()

reservation_id                 0
booking_date                   0
checkin_date                   0
checkout_date                  0
channel_code                   0
main_product_code              0
numberofadults                 0
numberofchildren               0
persontravellingid             0
resort_region_code             0
resort_type_code               0
room_type_booked_code          0
roomnights                     0
season_holidayed_code        149
state_code_residence        7024
state_code_resort              0
total_pax                      0
member_age_buckets             0
booking_type_code              0
memberid                       0
cluster_code                   0
reservationstatusid_code       0
resort_id                      0
dtype: int64

In [12]:
combi["season_holidayed_code"].fillna(combi["season_holidayed_code"].mean(), inplace=True)
combi["state_code_residence"].fillna(combi["state_code_residence"].mean(), inplace=True)

In [13]:
combi['booking_type_code'].value_counts()

1    427677
2     60512
Name: booking_type_code, dtype: int64

In [14]:
combi.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,...,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,...,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,...,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,...,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,...,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,...,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...


In [15]:
# creating and entity set 'es'
es = ft.EntitySet(id = 'amount_spent_per_room_night_scaled')

# adding a dataframe 
es.entity_from_dataframe(entity_id = 'fb', dataframe = combi, index = 'reservation_id')

Entityset: amount_spent_per_room_night_scaled
  Entities:
    fb [Rows: 488189, Columns: 23]
  Relationships:
    No relationships

In [16]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'fb', 
max_depth = 2, 
verbose = 1)

Built 33 features
Elapsed: 00:16 | Remaining: 00:00 | Progress: 100%|██████████████████████████████████████████| Calculated: 11/11 chunks


In [17]:
feature_matrix.columns

Index(['channel_code', 'main_product_code', 'numberofadults',
       'numberofchildren', 'persontravellingid', 'resort_region_code',
       'resort_type_code', 'room_type_booked_code', 'roomnights',
       'season_holidayed_code', 'state_code_residence', 'state_code_resort',
       'total_pax', 'member_age_buckets', 'booking_type_code', 'cluster_code',
       'reservationstatusid_code', 'DAY(booking_date)', 'DAY(checkin_date)',
       'DAY(checkout_date)', 'YEAR(booking_date)', 'YEAR(checkin_date)',
       'YEAR(checkout_date)', 'MONTH(booking_date)', 'MONTH(checkin_date)',
       'MONTH(checkout_date)', 'WEEKDAY(booking_date)',
       'WEEKDAY(checkin_date)', 'WEEKDAY(checkout_date)',
       'NUM_WORDS(memberid)', 'NUM_WORDS(resort_id)',
       'NUM_CHARACTERS(memberid)', 'NUM_CHARACTERS(resort_id)'],
      dtype='object')

In [18]:
feature_matrix = feature_matrix.reindex(index=combi['reservation_id'])
feature_matrix = feature_matrix.reset_index()

In [19]:
feature_matrix.head()

,reservation_id,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,...,MONTH(booking_date),MONTH(checkin_date),MONTH(checkout_date),WEEKDAY(booking_date),WEEKDAY(checkin_date),WEEKDAY(checkout_date),NUM_WORDS(memberid),NUM_WORDS(resort_id),NUM_CHARACTERS(memberid),NUM_CHARACTERS(resort_id)
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,3,1,2,0,46,3,3,3,1,...,5,5,6,4,4,0,1,1,64,64
1,03930f033646d073462b35d411616323597715ac4fc398...,1,1,2,0,46,3,3,4,5,...,1,11,4,4,2,3,1,1,64,64
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,1,1,2,0,47,1,5,4,4,...,1,1,5,2,4,5,1,1,64,64
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,1,1,2,2,46,2,2,3,5,...,2,11,6,3,4,1,1,1,64,64
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,1,1,2,0,46,2,2,4,5,...,2,12,12,0,0,5,1,1,64,64


In [20]:
#feature_matrix.to_csv("feature_matrix.csv")

In [21]:
feature_matrix.shape

(488189, 34)

In [22]:
feature_matrix.head()

,reservation_id,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,...,MONTH(booking_date),MONTH(checkin_date),MONTH(checkout_date),WEEKDAY(booking_date),WEEKDAY(checkin_date),WEEKDAY(checkout_date),NUM_WORDS(memberid),NUM_WORDS(resort_id),NUM_CHARACTERS(memberid),NUM_CHARACTERS(resort_id)
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,3,1,2,0,46,3,3,3,1,...,5,5,6,4,4,0,1,1,64,64
1,03930f033646d073462b35d411616323597715ac4fc398...,1,1,2,0,46,3,3,4,5,...,1,11,4,4,2,3,1,1,64,64
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,1,1,2,0,47,1,5,4,4,...,1,1,5,2,4,5,1,1,64,64
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,1,1,2,2,46,2,2,3,5,...,2,11,6,3,4,1,1,1,64,64
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,1,1,2,0,46,2,2,4,5,...,2,12,12,0,0,5,1,1,64,64


In [23]:
feature_matrix.drop(['reservation_id'], axis=1, inplace=True)
train = feature_matrix[:341424]
test = feature_matrix[341424:]

In [24]:
train.tail()

,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,...,MONTH(booking_date),MONTH(checkin_date),MONTH(checkout_date),WEEKDAY(booking_date),WEEKDAY(checkin_date),WEEKDAY(checkout_date),NUM_WORDS(memberid),NUM_WORDS(resort_id),NUM_CHARACTERS(memberid),NUM_CHARACTERS(resort_id)
341419,2,2,4,0,47,1,1,2,4,4.0,...,11,2,1,5,2,3,1,1,64,64
341420,2,4,2,0,45,1,5,3,1,4.0,...,2,2,1,2,3,3,1,1,64,64
341421,3,3,3,0,45,1,1,3,2,4.0,...,2,2,2,2,2,3,1,1,64,64
341422,1,3,5,0,47,2,2,2,2,3.0,...,2,2,1,2,3,3,1,1,64,64
341423,3,3,2,0,45,2,1,2,1,2.0,...,2,1,2,3,3,6,1,1,64,64


In [25]:
train.shape

(341424, 33)

In [26]:
train= pd.concat([train,amount_spent_per_room_night_scaled ],axis=1)

In [27]:
train.to_csv("newtrain.csv")
test.to_csv("newtest.csv")

In [28]:
train['Type']='Train'
test['Type']='Test'
data = pd.concat([train,test],axis=0)

C:\Users\AJIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
data.head()

,DAY(booking_date),DAY(checkin_date),DAY(checkout_date),MONTH(booking_date),MONTH(checkin_date),MONTH(checkout_date),NUM_CHARACTERS(memberid),NUM_CHARACTERS(resort_id),NUM_WORDS(memberid),NUM_WORDS(resort_id),...,persontravellingid,reservationstatusid_code,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax
0,4,4,4,5,5,6,64,64,1,1,...,46,C,3,3,3,1,2.0,7.0,3,3
1,23,4,16,1,11,4,64,64,1,1,...,46,A,3,3,4,5,2.0,7.0,5,2
2,28,2,2,1,1,5,64,64,1,1,...,47,A,1,5,4,4,2.0,7.0,1,2
3,5,6,16,2,11,6,64,64,1,1,...,46,A,2,2,3,5,2.0,7.0,2,2
4,9,14,19,2,12,12,64,64,1,1,...,46,A,2,2,4,5,2.0,7.0,2,2


In [30]:
data.columns

Index(['DAY(booking_date)', 'DAY(checkin_date)', 'DAY(checkout_date)',
       'MONTH(booking_date)', 'MONTH(checkin_date)', 'MONTH(checkout_date)',
       'NUM_CHARACTERS(memberid)', 'NUM_CHARACTERS(resort_id)',
       'NUM_WORDS(memberid)', 'NUM_WORDS(resort_id)', 'Type',
       'WEEKDAY(booking_date)', 'WEEKDAY(checkin_date)',
       'WEEKDAY(checkout_date)', 'YEAR(booking_date)', 'YEAR(checkin_date)',
       'YEAR(checkout_date)', 'amount_spent_per_room_night_scaled',
       'booking_type_code', 'channel_code', 'cluster_code',
       'main_product_code', 'member_age_buckets', 'numberofadults',
       'numberofchildren', 'persontravellingid', 'reservationstatusid_code',
       'resort_region_code', 'resort_type_code', 'room_type_booked_code',
       'roomnights', 'season_holidayed_code', 'state_code_residence',
       'state_code_resort', 'total_pax'],
      dtype='object')

In [31]:
#data['total_pax'].value_counts()

In [32]:
data['YEAR(booking_date)']=data['YEAR(booking_date)'].astype('str')
data['YEAR(checkin_date)']=data['YEAR(checkin_date)'].astype('str')
data['YEAR(checkout_date)']=data['YEAR(checkout_date)'].astype('str')
data['booking_type_code']=data['booking_type_code'].astype('str')
data['channel_code']=data['channel_code'].astype('str')
data['cluster_code']=data['cluster_code'].astype('str')
data['main_product_code']=data['main_product_code'].astype('str')
data['member_age_buckets']=data['member_age_buckets'].astype('str')
data['numberofchildren']=data['numberofchildren'].astype('str')
data['persontravellingid']=data['persontravellingid'].astype('str')
data['reservationstatusid_code']=data['reservationstatusid_code'].astype('str')
data['resort_region_code']=data['resort_region_code'].astype('str')
data['resort_type_code']=data['resort_type_code'].astype('str')
data['room_type_booked_code']=data['room_type_booked_code'].astype('str')
data['season_holidayed_code']=data['season_holidayed_code'].astype('str')
data['state_code_resort']=data['state_code_resort'].astype('str')

In [33]:
cat_var = [key for key in dict(data.dtypes)
             if dict(data.dtypes)[key] in ['object'] ]
cat_var.remove("Type")
cat_var

['YEAR(booking_date)',
 'YEAR(checkin_date)',
 'YEAR(checkout_date)',
 'booking_type_code',
 'channel_code',
 'cluster_code',
 'main_product_code',
 'member_age_buckets',
 'numberofchildren',
 'persontravellingid',
 'reservationstatusid_code',
 'resort_region_code',
 'resort_type_code',
 'room_type_booked_code',
 'season_holidayed_code',
 'state_code_resort']

In [34]:
from sklearn.preprocessing import LabelEncoder
data[cat_var]=data[cat_var].apply(LabelEncoder().fit_transform)

In [37]:
data=pd.concat([data,pd.get_dummies(data['YEAR(booking_date)'],prefix='YEAR(booking_date)')],axis=1).drop(['YEAR(booking_date)'],axis=1)
data=pd.concat([data,pd.get_dummies(data['YEAR(checkin_date)'],prefix='YEAR(checkin_date)')],axis=1).drop(['YEAR(checkin_date)'],axis=1)
data=pd.concat([data,pd.get_dummies(data['YEAR(checkout_date)'],prefix='YEAR(checkout_date)')],axis=1).drop(['YEAR(checkout_date)'],axis=1)
data=pd.concat([data,pd.get_dummies(data['booking_type_code'],prefix='booking_type_code')],axis=1).drop(['booking_type_code'],axis=1)
data=pd.concat([data,pd.get_dummies(data['channel_code'],prefix='channel_code')],axis=1).drop(['channel_code'],axis=1)
data=pd.concat([data,pd.get_dummies(data['cluster_code'],prefix='cluster_code')],axis=1).drop(['cluster_code'],axis=1)
data=pd.concat([data,pd.get_dummies(data['main_product_code'],prefix='main_product_code')],axis=1).drop(['main_product_code'],axis=1)
data=pd.concat([data,pd.get_dummies(data['member_age_buckets'],prefix='member_age_buckets')],axis=1).drop(['member_age_buckets'],axis=1)
data=pd.concat([data,pd.get_dummies(data['numberofchildren'],prefix='numberofchildren')],axis=1).drop(['numberofchildren'],axis=1)
data=pd.concat([data,pd.get_dummies(data['persontravellingid'],prefix='persontravellingid')],axis=1).drop(['persontravellingid'],axis=1)
data=pd.concat([data,pd.get_dummies(data['reservationstatusid_code'],prefix='reservationstatusid_code')],axis=1).drop(['reservationstatusid_code'],axis=1)
data=pd.concat([data,pd.get_dummies(data['resort_region_code'],prefix='resort_region_code')],axis=1).drop(['resort_region_code'],axis=1)
data=pd.concat([data,pd.get_dummies(data['resort_type_code'],prefix='resort_type_code')],axis=1).drop(['resort_type_code'],axis=1)
data=pd.concat([data,pd.get_dummies(data['room_type_booked_code'],prefix='room_type_booked_code')],axis=1).drop(['room_type_booked_code'],axis=1)
data=pd.concat([data,pd.get_dummies(data['season_holidayed_code'],prefix='season_holidayed_code')],axis=1).drop(['season_holidayed_code'],axis=1)
data=pd.concat([data,pd.get_dummies(data['state_code_resort'],prefix='state_code_resort')],axis=1).drop(['state_code_resort'],axis=1)

In [38]:
data.head()

,DAY(booking_date),DAY(checkin_date),DAY(checkout_date),MONTH(booking_date),MONTH(checkin_date),MONTH(checkout_date),NUM_CHARACTERS(memberid),NUM_CHARACTERS(resort_id),NUM_WORDS(memberid),NUM_WORDS(resort_id),...,state_code_resort_1,state_code_resort_2,state_code_resort_3,state_code_resort_4,state_code_resort_5,state_code_resort_6,state_code_resort_7,state_code_resort_8,state_code_resort_9,state_code_resort_10
0,4,4,4,5,5,6,64,64,1,1,...,0,0,0,0,1,0,0,0,0,0
1,23,4,16,1,11,4,64,64,1,1,...,0,0,0,0,0,0,1,0,0,0
2,28,2,2,1,1,5,64,64,1,1,...,0,0,0,0,0,0,0,0,0,0
3,5,6,16,2,11,6,64,64,1,1,...,0,0,0,1,0,0,0,0,0,0
4,9,14,19,2,12,12,64,64,1,1,...,0,0,0,1,0,0,0,0,0,0


In [50]:
#Id_cols=["reservation_id","resort_id","memberid"]
target_col = ["amount_spent_per_room_night_scaled"]
#cat_cols = cat_var
other_col=['Type','NUM_CHARACTERS(memberid)','NUM_CHARACTERS(resort_id)',
          "NUM_WORDS(memberid)",'NUM_WORDS(resort_id)'] #Test and Train Data set identifier

#num_cols= list(set(list(data.columns))-set(cat_cols)-set(target_col)-set(other_col))

In [51]:
train=data[data['Type']=='Train']
test=data[data['Type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]

In [52]:
features=list(set(list(data.columns))-set(target_col)-set(other_col))

In [53]:
x_train = Train[list(features)]
y_train = Train["amount_spent_per_room_night_scaled"]
x_validate = Validate[list(features)]
y_validate = Validate["amount_spent_per_room_night_scaled"]
x_test=test[list(features)]

In [54]:
x_train.shape

(255522, 112)

In [56]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_validate = sc.transform(x_validate)

C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\AJIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
C:\Users\AJIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  """


In [45]:
x_train.shape

(255881, 112)

In [55]:
df=pd.concat([x_train,y_train],axis=1)

### RFECV

In [309]:
#https://chrisalbon.com/machine_learning/feature_selection/recursive_feature_elimination/
#https://www.scikit-yb.org/en/latest/api/features/rfecv.html
from sklearn.feature_selection import RFECV
from sklearn import linear_model

In [310]:
ols = linear_model.LinearRegression()
rfecv = RFECV(estimator=ols, step=1, scoring='neg_mean_squared_error')

In [313]:
rfecv.fit(x_train, y_train)

C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


RFECV(cv='warn',
   estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
   min_features_to_select=1, n_jobs=None, scoring='neg_mean_squared_error',
   step=1, verbose=0)

In [314]:
x_train=rfecv.transform(x_train)
x_validate=rfecv.transform(x_validate)
x_test=rfecv.transform(x_test)

In [315]:
x_train.shape

(256080, 32)

### Catboost

In [316]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=2000)
model.fit(x_train,y_train)

0:	learn: 7.4153915	total: 276ms	remaining: 9m 11s
1:	learn: 7.0528320	total: 367ms	remaining: 6m 6s
2:	learn: 6.7087824	total: 491ms	remaining: 5m 27s
3:	learn: 6.3823424	total: 621ms	remaining: 5m 9s
4:	learn: 6.0726974	total: 742ms	remaining: 4m 55s
5:	learn: 5.7789792	total: 846ms	remaining: 4m 41s
6:	learn: 5.5004632	total: 944ms	remaining: 4m 28s
7:	learn: 5.2363742	total: 1.06s	remaining: 4m 25s
8:	learn: 4.9859695	total: 1.17s	remaining: 4m 19s
9:	learn: 4.7487800	total: 1.26s	remaining: 4m 10s
10:	learn: 4.5240320	total: 1.36s	remaining: 4m 5s
11:	learn: 4.3111042	total: 1.48s	remaining: 4m 4s
12:	learn: 4.1094680	total: 1.58s	remaining: 4m 1s
13:	learn: 3.9186365	total: 1.69s	remaining: 3m 59s
14:	learn: 3.7380631	total: 1.78s	remaining: 3m 55s
15:	learn: 3.5672017	total: 1.85s	remaining: 3m 50s
16:	learn: 3.4056919	total: 1.96s	remaining: 3m 48s
17:	learn: 3.2530307	total: 2.11s	remaining: 3m 52s
18:	learn: 3.1087784	total: 2.2s	remaining: 3m 49s
19:	learn: 2.9725653	total: 

158:	learn: 1.0737508	total: 16.6s	remaining: 3m 12s
159:	learn: 1.0737351	total: 16.7s	remaining: 3m 12s
160:	learn: 1.0737316	total: 16.8s	remaining: 3m 12s
161:	learn: 1.0737230	total: 16.9s	remaining: 3m 11s
162:	learn: 1.0737169	total: 17s	remaining: 3m 11s
163:	learn: 1.0737085	total: 17.1s	remaining: 3m 11s
164:	learn: 1.0736966	total: 17.2s	remaining: 3m 11s
165:	learn: 1.0736839	total: 17.3s	remaining: 3m 11s
166:	learn: 1.0736764	total: 17.4s	remaining: 3m 11s
167:	learn: 1.0736703	total: 17.5s	remaining: 3m 11s
168:	learn: 1.0736666	total: 17.6s	remaining: 3m 10s
169:	learn: 1.0736633	total: 17.7s	remaining: 3m 10s
170:	learn: 1.0736490	total: 17.8s	remaining: 3m 10s
171:	learn: 1.0736455	total: 17.9s	remaining: 3m 10s
172:	learn: 1.0736384	total: 18s	remaining: 3m 10s
173:	learn: 1.0736272	total: 18.1s	remaining: 3m 10s
174:	learn: 1.0736243	total: 18.2s	remaining: 3m 9s
175:	learn: 1.0736212	total: 18.3s	remaining: 3m 9s
176:	learn: 1.0736159	total: 18.4s	remaining: 3m 9s


315:	learn: 1.0731813	total: 32s	remaining: 2m 50s
316:	learn: 1.0731801	total: 32.1s	remaining: 2m 50s
317:	learn: 1.0731783	total: 32.2s	remaining: 2m 50s
318:	learn: 1.0731759	total: 32.3s	remaining: 2m 50s
319:	learn: 1.0731747	total: 32.4s	remaining: 2m 50s
320:	learn: 1.0731741	total: 32.5s	remaining: 2m 50s
321:	learn: 1.0731730	total: 32.6s	remaining: 2m 49s
322:	learn: 1.0731715	total: 32.7s	remaining: 2m 49s
323:	learn: 1.0731691	total: 32.8s	remaining: 2m 49s
324:	learn: 1.0731675	total: 32.9s	remaining: 2m 49s
325:	learn: 1.0731652	total: 33s	remaining: 2m 49s
326:	learn: 1.0731633	total: 33.1s	remaining: 2m 49s
327:	learn: 1.0731623	total: 33.2s	remaining: 2m 49s
328:	learn: 1.0731600	total: 33.3s	remaining: 2m 49s
329:	learn: 1.0731581	total: 33.4s	remaining: 2m 49s
330:	learn: 1.0731574	total: 33.5s	remaining: 2m 48s
331:	learn: 1.0731548	total: 33.6s	remaining: 2m 48s
332:	learn: 1.0731531	total: 33.7s	remaining: 2m 48s
333:	learn: 1.0731525	total: 33.8s	remaining: 2m 4

471:	learn: 1.0730131	total: 48s	remaining: 2m 35s
472:	learn: 1.0730126	total: 48.1s	remaining: 2m 35s
473:	learn: 1.0730122	total: 48.2s	remaining: 2m 35s
474:	learn: 1.0730113	total: 48.4s	remaining: 2m 35s
475:	learn: 1.0730100	total: 48.5s	remaining: 2m 35s
476:	learn: 1.0730086	total: 48.6s	remaining: 2m 35s
477:	learn: 1.0730083	total: 48.8s	remaining: 2m 35s
478:	learn: 1.0730078	total: 48.9s	remaining: 2m 35s
479:	learn: 1.0730067	total: 49s	remaining: 2m 35s
480:	learn: 1.0730055	total: 49.1s	remaining: 2m 34s
481:	learn: 1.0730032	total: 49.1s	remaining: 2m 34s
482:	learn: 1.0730013	total: 49.3s	remaining: 2m 34s
483:	learn: 1.0730008	total: 49.4s	remaining: 2m 34s
484:	learn: 1.0729997	total: 49.4s	remaining: 2m 34s
485:	learn: 1.0729996	total: 49.5s	remaining: 2m 34s
486:	learn: 1.0729984	total: 49.6s	remaining: 2m 34s
487:	learn: 1.0729975	total: 49.7s	remaining: 2m 34s
488:	learn: 1.0729967	total: 49.8s	remaining: 2m 34s
489:	learn: 1.0729962	total: 49.9s	remaining: 2m 3

627:	learn: 1.0729232	total: 1m 3s	remaining: 2m 18s
628:	learn: 1.0729232	total: 1m 3s	remaining: 2m 18s
629:	learn: 1.0729230	total: 1m 3s	remaining: 2m 18s
630:	learn: 1.0729226	total: 1m 3s	remaining: 2m 18s
631:	learn: 1.0729210	total: 1m 3s	remaining: 2m 18s
632:	learn: 1.0729209	total: 1m 4s	remaining: 2m 18s
633:	learn: 1.0729206	total: 1m 4s	remaining: 2m 18s
634:	learn: 1.0729205	total: 1m 4s	remaining: 2m 18s
635:	learn: 1.0729201	total: 1m 4s	remaining: 2m 17s
636:	learn: 1.0729199	total: 1m 4s	remaining: 2m 17s
637:	learn: 1.0729197	total: 1m 4s	remaining: 2m 17s
638:	learn: 1.0729192	total: 1m 4s	remaining: 2m 17s
639:	learn: 1.0729191	total: 1m 4s	remaining: 2m 17s
640:	learn: 1.0729189	total: 1m 4s	remaining: 2m 17s
641:	learn: 1.0729183	total: 1m 4s	remaining: 2m 17s
642:	learn: 1.0729177	total: 1m 5s	remaining: 2m 17s
643:	learn: 1.0729175	total: 1m 5s	remaining: 2m 17s
644:	learn: 1.0729170	total: 1m 5s	remaining: 2m 17s
645:	learn: 1.0729160	total: 1m 5s	remaining: 

...

1237:	learn: 1.0727962	total: 2m 5s	remaining: 1m 17s
1238:	learn: 1.0727960	total: 2m 5s	remaining: 1m 17s
1239:	learn: 1.0727960	total: 2m 6s	remaining: 1m 17s
1240:	learn: 1.0727960	total: 2m 6s	remaining: 1m 17s
1241:	learn: 1.0727960	total: 2m 6s	remaining: 1m 17s
1242:	learn: 1.0727959	total: 2m 6s	remaining: 1m 16s
1243:	learn: 1.0727958	total: 2m 6s	remaining: 1m 16s
1244:	learn: 1.0727957	total: 2m 6s	remaining: 1m 16s
1245:	learn: 1.0727957	total: 2m 6s	remaining: 1m 16s
1246:	learn: 1.0727956	total: 2m 6s	remaining: 1m 16s
1247:	learn: 1.0727956	total: 2m 6s	remaining: 1m 16s
1248:	learn: 1.0727954	total: 2m 6s	remaining: 1m 16s
1249:	learn: 1.0727953	total: 2m 7s	remaining: 1m 16s
1250:	learn: 1.0727952	total: 2m 7s	remaining: 1m 16s
1251:	learn: 1.0727949	total: 2m 7s	remaining: 1m 16s
1252:	learn: 1.0727948	total: 2m 7s	remaining: 1m 15s
1253:	learn: 1.0727946	total: 2m 7s	remaining: 1m 15s
1254:	learn: 1.0727945	total: 2m 7s	remaining: 1m 15s
1255:	learn: 1.0727945	total

1389:	learn: 1.0727816	total: 2m 20s	remaining: 1m 1s
1390:	learn: 1.0727814	total: 2m 21s	remaining: 1m 1s
1391:	learn: 1.0727812	total: 2m 21s	remaining: 1m 1s
1392:	learn: 1.0727812	total: 2m 21s	remaining: 1m 1s
1393:	learn: 1.0727812	total: 2m 21s	remaining: 1m 1s
1394:	learn: 1.0727810	total: 2m 21s	remaining: 1m 1s
1395:	learn: 1.0727810	total: 2m 21s	remaining: 1m 1s
1396:	learn: 1.0727809	total: 2m 21s	remaining: 1m 1s
1397:	learn: 1.0727809	total: 2m 21s	remaining: 1m 1s
1398:	learn: 1.0727808	total: 2m 21s	remaining: 1m
1399:	learn: 1.0727807	total: 2m 21s	remaining: 1m
1400:	learn: 1.0727806	total: 2m 21s	remaining: 1m
1401:	learn: 1.0727805	total: 2m 22s	remaining: 1m
1402:	learn: 1.0727805	total: 2m 22s	remaining: 1m
1403:	learn: 1.0727803	total: 2m 22s	remaining: 1m
1404:	learn: 1.0727803	total: 2m 22s	remaining: 1m
1405:	learn: 1.0727802	total: 2m 22s	remaining: 1m
1406:	learn: 1.0727801	total: 2m 22s	remaining: 1m
1407:	learn: 1.0727800	total: 2m 22s	remaining: 60s
140

1542:	learn: 1.0727688	total: 2m 35s	remaining: 46.1s
1543:	learn: 1.0727686	total: 2m 35s	remaining: 46s
1544:	learn: 1.0727685	total: 2m 36s	remaining: 46s
1545:	learn: 1.0727685	total: 2m 36s	remaining: 45.9s
1546:	learn: 1.0727684	total: 2m 36s	remaining: 45.8s
1547:	learn: 1.0727684	total: 2m 36s	remaining: 45.7s
1548:	learn: 1.0727684	total: 2m 36s	remaining: 45.6s
1549:	learn: 1.0727684	total: 2m 36s	remaining: 45.5s
1550:	learn: 1.0727683	total: 2m 36s	remaining: 45.4s
1551:	learn: 1.0727681	total: 2m 36s	remaining: 45.3s
1552:	learn: 1.0727680	total: 2m 37s	remaining: 45.2s
1553:	learn: 1.0727680	total: 2m 37s	remaining: 45.1s
1554:	learn: 1.0727679	total: 2m 37s	remaining: 45s
1555:	learn: 1.0727679	total: 2m 37s	remaining: 44.9s
1556:	learn: 1.0727679	total: 2m 37s	remaining: 44.8s
1557:	learn: 1.0727677	total: 2m 37s	remaining: 44.7s
1558:	learn: 1.0727677	total: 2m 37s	remaining: 44.6s
1559:	learn: 1.0727676	total: 2m 37s	remaining: 44.5s
1560:	learn: 1.0727676	total: 2m 3

1695:	learn: 1.0727598	total: 2m 51s	remaining: 30.8s
1696:	learn: 1.0727596	total: 2m 51s	remaining: 30.7s
1697:	learn: 1.0727595	total: 2m 52s	remaining: 30.6s
1698:	learn: 1.0727592	total: 2m 52s	remaining: 30.5s
1699:	learn: 1.0727592	total: 2m 52s	remaining: 30.4s
1700:	learn: 1.0727592	total: 2m 52s	remaining: 30.3s
1701:	learn: 1.0727592	total: 2m 52s	remaining: 30.2s
1702:	learn: 1.0727591	total: 2m 52s	remaining: 30.1s
1703:	learn: 1.0727590	total: 2m 52s	remaining: 30s
1704:	learn: 1.0727590	total: 2m 52s	remaining: 29.9s
1705:	learn: 1.0727589	total: 2m 52s	remaining: 29.8s
1706:	learn: 1.0727589	total: 2m 52s	remaining: 29.7s
1707:	learn: 1.0727589	total: 2m 52s	remaining: 29.6s
1708:	learn: 1.0727588	total: 2m 53s	remaining: 29.5s
1709:	learn: 1.0727588	total: 2m 53s	remaining: 29.4s
1710:	learn: 1.0727588	total: 2m 53s	remaining: 29.3s
1711:	learn: 1.0727587	total: 2m 53s	remaining: 29.2s
1712:	learn: 1.0727585	total: 2m 53s	remaining: 29.1s
1713:	learn: 1.0727584	total: 

1850:	learn: 1.0727502	total: 3m 8s	remaining: 15.2s
1851:	learn: 1.0727501	total: 3m 9s	remaining: 15.1s
1852:	learn: 1.0727501	total: 3m 9s	remaining: 15s
1853:	learn: 1.0727500	total: 3m 9s	remaining: 14.9s
1854:	learn: 1.0727500	total: 3m 9s	remaining: 14.8s
1855:	learn: 1.0727500	total: 3m 9s	remaining: 14.7s
1856:	learn: 1.0727500	total: 3m 9s	remaining: 14.6s
1857:	learn: 1.0727500	total: 3m 9s	remaining: 14.5s
1858:	learn: 1.0727499	total: 3m 9s	remaining: 14.4s
1859:	learn: 1.0727499	total: 3m 9s	remaining: 14.3s
1860:	learn: 1.0727499	total: 3m 10s	remaining: 14.2s
1861:	learn: 1.0727498	total: 3m 10s	remaining: 14.1s
1862:	learn: 1.0727498	total: 3m 10s	remaining: 14s
1863:	learn: 1.0727498	total: 3m 10s	remaining: 13.9s
1864:	learn: 1.0727498	total: 3m 10s	remaining: 13.8s
1865:	learn: 1.0727498	total: 3m 10s	remaining: 13.7s
1866:	learn: 1.0727498	total: 3m 10s	remaining: 13.6s
1867:	learn: 1.0727498	total: 3m 10s	remaining: 13.5s
1868:	learn: 1.0727497	total: 3m 10s	remai

In [317]:
# calculating RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
pred = model.predict(x_validate)
rmse = sqrt(mean_squared_error(y_validate, pred))
rmse*100

107.71908141642

In [312]:
pred=model.predict(x_test)
test["amount_spent_per_room_night_scaled"]=pred
test.to_csv("cat.csv",columns=['reservation_id','amount_spent_per_room_night_scaled'])

C:\Users\AJIT\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


### Random Forest

In [318]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(random_state=0,n_estimators=100)
regr.fit(x_train, y_train)

KeyboardInterrupt: 

In [140]:
import math
pred = regr.predict(x_validate)
mse = np.mean((pred - y_validate)**2)
mse

1.0814306074848268

### GBM

In [41]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators=3000)
gbm.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=3000, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [44]:
# calculating RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
pred = gbm.predict(x_validate)
rmse = sqrt(mean_squared_error(y_validate, pred)
rmse

In [43]:
pred=gbm.predict(x_test)
test["amount_spent_per_room_night_scaled"]=pred
test.to_csv("gbm.csv",columns=['reservation_id','amount_spent_per_room_night_scaled'])

C:\Users\AJIT\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


### XgBoost

In [51]:
from xgboost import XGBRegressor
model= XGBRegressor(n_estimators=500)
model.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [53]:
# calculating RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
pred = model.predict(x_validate)
rmse = sqrt(mean_squared_error(y_validate, pred))
rmse*100

100.43930139682493

### LightGBM

In [70]:
from lightgbm import LGBMRegressor
model= LGBMRegressor(n_estimators=1000)
model.fit(x_train,y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [71]:
pred = model.predict(x_validate)
mse = np.mean((pred - y_validate)**2)
mse*100

98.08966965739138

In [72]:
pred=model.predict(x_test)
test["amount_spent_per_room_night_scaled"]=pred
test.to_csv("lightgbm.csv",columns=['reservation_id','amount_spent_per_room_night_scaled'])

C:\Users\AJIT\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


### AdaBoost

In [75]:
from sklearn.ensemble import AdaBoostRegressor
ada= AdaBoostRegressor(n_estimators=500)
ada.fit(x_train,y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=500, random_state=None)

In [76]:
pred = ada.predict(x_validate)
mse = np.mean((pred - y_validate)**2)
mse*100

136.03022592961378

### Stacking

In [46]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [47]:
SEED=10
def get_models():
    """Generate a library of base learners."""
    cat = CatBoostRegressor(n_estimators=2500)
    xg=XGBRegressor(n_estimators=500)
    #lgbm =LGBMRegressor(silent=False,n_estimators=500)
    ada = AdaBoostRegressor(n_estimators=500)
    gbm = GradientBoostingRegressor(n_estimators=2000, random_state=SEED)
    models = {'CATBOOST': cat,
              'XGBOOST': xg,
             # 'LIGHTGBM': lgbm,
              'ADABOOST': ada,
              'GBM': gbm,
              }

    return models

In [48]:
base_learners = get_models()

In [49]:
meta_learner =  LGBMRegressor(n_estimators=2000, random_state=SEED)

In [50]:
from mlens.ensemble import SuperLearner
# Instantiate the ensemble with 10 folds
sl = SuperLearner(
    folds=3,
    random_state=SEED,
    verbose=2,
    backend="multiprocessing")

In [51]:
sl.add(list(base_learners.values()))
sl.add_meta(meta_learner)

SuperLearner(array_check=None, backend=None, folds=3,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=1289, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=3, raise_on_ex...rer=None)],
   n_jobs=-1, name='group-3', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=10, sample_size=20, scorer=None, shuffle=False,
       verbose=2)

In [52]:
# Train the ensemble
sl.fit(x_train, y_train)


Fitting 2 layers
Processing layer-1             done | 01:16:09
Processing layer-2             done | 00:01:07
Fit complete                        | 01:17:19


SuperLearner(array_check=None, backend=None, folds=3,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=1289, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=3, raise_on_ex...rer=None)],
   n_jobs=-1, name='group-3', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=10, sample_size=20, scorer=None, shuffle=False,
       verbose=2)

In [53]:
pred = sl.predict(x_validate)
mse = np.mean((pred - y_validate)**2)
mse*100


Predicting 2 layers
Processing layer-1             done | 00:00:31
Processing layer-2             done | 00:00:07
Predict complete                    | 00:00:39


102.02643484649394

In [45]:
pred=sl.predict(x_test)
test["amount_spent_per_room_night_scaled"]=pred
test.to_csv("stacking.csv",columns=['reservation_id','amount_spent_per_room_night_scaled'])


Predicting 2 layers
Processing layer-1             done | 00:00:13
Processing layer-2             done | 00:00:49
Predict complete                    | 00:01:04


C:\Users\AJIT\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


### AUTOML

In [36]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,14 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.4
H2O cluster version age:,2 months and 21 days
H2O cluster name:,H2O_from_python_AJIT_8lo5cw
H2O cluster total nodes:,1
H2O cluster free memory:,877 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [37]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("newtrain.csv")
test = h2o.import_file("newtest.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [39]:
# Identify predictors and response
x = train.columns
y = "amount_spent_per_room_night_scaled"
x.remove(y)

In [40]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [41]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20190507_093006,0.999081,0.99954,0.999081,0.732077,0.12854
StackedEnsemble_BestOfFamily_AutoML_20190507_093006,1.00063,1.00031,1.00063,0.732867,0.128628
GBM_1_AutoML_20190507_093006,1.0014,1.0007,1.0014,0.733683,0.128677
GBM_3_AutoML_20190507_093006,1.00694,1.00346,1.00694,0.735665,0.128933
GBM_2_AutoML_20190507_093006,1.00789,1.00394,1.00789,0.735818,0.129005
GLM_grid_1_AutoML_20190507_093006_model_1,1.09714,1.04744,1.09714,0.772413,0.13368


In [42]:
# The leader model is stored here
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20190507_093006
No model summary for this model


ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.9370590511435961
RMSE: 0.9680181047602344
MAE: 0.7101899418642309
RMSLE: 0.12499570686341907
R^2: 0.21145174667036237
Mean Residual Deviance: 0.9370590511435961
Null degrees of freedom: 341423
Residual degrees of freedom: 341420
Null deviance: 405725.89962152357
Residual deviance: 319934.44947765116
AIC: 946733.7144794506

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.9990810096280076
RMSE: 0.999540399197555
MAE: 0.7320770040073814
RMSLE: 0.12853971224142935
R^2: 0.15925940407217287
Mean Residual Deviance: 0.9990810096280076
Null degrees of freedom: 341423
Residual degrees of freedom: 341420
Null deviance: 405728.4193129648
Residual deviance: 341110.23463123286
AIC: 968615.4298909104


In [43]:
preds = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [44]:
preds

predict
8.13616
7.51814
7.77396
7.95776
7.87966
8.0503
7.27957
8.61024
8.12942
7.94676
